# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.5667,143.2000,-13.16,94,100,1.37,RU,1697320504
1,1,jining,35.4050,116.5814,14.22,49,0,2.51,CN,1697320504
2,2,verkhnevilyuysk,63.4453,120.3167,-11.14,92,5,2.24,RU,1697320504
3,3,kodiak,57.7900,-152.4072,6.73,70,0,2.06,US,1697320504
4,4,ola,59.5833,151.2833,-1.44,68,94,1.92,RU,1697320505


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"    
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_updated_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
city_data_cleaned_df = city_data_updated_df.dropna() 

# Display sample data
city_data_cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,limassol,34.6750,33.0333,23.04,68,0,4.12,CY,1697320513
64,64,sirte,31.2089,16.5887,23.12,70,0,1.50,LY,1697320516
68,68,al kharijah,25.4514,30.5464,23.83,34,0,2.89,EG,1697320517
132,132,ghadamis,30.1337,9.5007,23.46,22,0,3.49,LY,1697320531
135,135,al jawf,29.5000,38.7500,22.62,21,0,3.78,SA,1697320531
146,146,al khums,32.6486,14.2619,22.85,66,0,0.10,LY,1697320534
181,181,nefta,33.8731,7.8777,26.14,25,0,2.99,TN,1697320541
192,192,kempsey,-31.0833,152.8333,21.64,59,0,1.79,AU,1697320542
282,282,arrecife,28.9630,-13.5477,24.99,69,0,0.00,ES,1697320559
333,333,kantilo,20.3500,85.1833,23.12,80,0,1.36,IN,1697320569


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_cleaned_df[["City","Country","Lat","Lng","Humidity"]]
rows= len(hotel_df["City"])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\Vinaya Kusuma\AppData\Local\Temp\ipykernel_33980\578753333.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
51,limassol,CY,34.6750,33.0333,68,
64,sirte,LY,31.2089,16.5887,70,
68,al kharijah,EG,25.4514,30.5464,34,
132,ghadamis,LY,30.1337,9.5007,22,
135,al jawf,SA,29.5000,38.7500,21,
146,al khums,LY,32.6486,14.2619,66,
181,nefta,TN,33.8731,7.8777,25,
192,kempsey,AU,-31.0833,152.8333,59,
282,arrecife,ES,28.9630,-13.5477,69,
333,kantilo,IN,20.3500,85.1833,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "apiKey":geoapify_key    
}

params = {
    "apiKey":geoapify_key,
    "limit":limit 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    # YOUR CODE HERE
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = filters
    params["bias"] = bias
    params["categories"] = f"catering.restaurant"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
limassol - nearest hotel: To Mageirio
sirte - nearest hotel: مطعم الشراع للوجبات البحريه
al kharijah - nearest hotel: Wimby
ghadamis - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
al khums - nearest hotel: Pizza time
nefta - nearest hotel: La Source
kempsey - nearest hotel: Hong Kong Chinese Restaurant
arrecife - nearest hotel: naia
kantilo - nearest hotel: No hotel found
maceio - nearest hotel: Restaurante Zepellin
darab - nearest hotel: city food
saint-pierre - nearest hotel: Le Binôme
winnemucca - nearest hotel: Le French Twiste
narrabri - nearest hotel: One23 Cafe & Restaurant
megarine - nearest hotel: Resto
amta - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
ghardaia - nearest hotel: Resto
siwa oasis - nearest hotel: مطعم شهرزاد


,City,Country,Lat,Lng,Humidity,Hotel Name
51,limassol,CY,34.6750,33.0333,68,To Mageirio
64,sirte,LY,31.2089,16.5887,70,مطعم الشراع للوجبات البحريه
68,al kharijah,EG,25.4514,30.5464,34,Wimby
132,ghadamis,LY,30.1337,9.5007,22,No hotel found
135,al jawf,SA,29.5000,38.7500,21,No hotel found
146,al khums,LY,32.6486,14.2619,66,Pizza time
181,nefta,TN,33.8731,7.8777,25,La Source
192,kempsey,AU,-31.0833,152.8333,59,Hong Kong Chinese Restaurant
282,arrecife,ES,28.9630,-13.5477,69,naia
333,kantilo,IN,20.3500,85.1833,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols=['Lng','Lat','City','Humidity', 'Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)